## 3. Many-body Ordered phases

Let's extend the analysis to many qubits. We will start this session by building up our intuition with an exercise straight away:

In [ ]:
FreeResponseQuestion("exercise23")

Welcome to the realm of simulating ordered phases of matter with neutral-atom quantum computers! By positioning qubits in different lattice structures, we can leverage the natural Rydberg interaction to create organizational patterns found in nature such as anti-ferromagnets and more exotic systems. The preparation of these patterns often involves a quantum dynamic process, allowing us to further simulate how different patterns emerge in a time-dependent fashion. One can also study how perturbations in these patterns evolve and change character. Some of these experiments, at the largest possible scales achieved with Aquila (256 qubits), can become extremely strenuous to classical computers.

To set up the paradigm, this session will explore a bit the space of ordered phases in 2D square lattices and a specific protocol for creating these phases: the adiabatic protocol. This will be your first official engagement with a quantum heuristic algorithm in this course!

### 3a. The square lattice Rydberg phase diagram

This may be your first interaction with what in physics is called a "phase diagram". A quantum one for that matter! This type of diagram describes the different ordering structures that an interacting many-body system may spontaneously display as function of parameters that controls its energy or other physical properties.

Let's look at what a phase diagram for our standard Rydberg atom system looks like when atoms are positioned in a regular square lattice:

<img src="./assets/square_phasediagram.png" width="700" height="auto" />

This figure is adapted from the following reference: R. Samajdar et al., Phys. Rev. Lett. 124, 103601 (2020). On the left, we see the atomic square lattice, while the center figure is our phase diagram; insets on the right point to different Rydberg excitation patterns corresponding to each of the phases in the center diagram.

In the phase diagram, the $x$ and $y$ axes are given by, respectively, the ratio of the detuning field to the Rabi amplitude (in our notation in this course, substitute $\delta \to \Delta$), and the ratio of the blockade radius defined by the Rabi amplitude $ R_B= (C_6⁡/\Omega)^{1/6}$ over the square lattice spacing size $a$. The blue scale corresponds to a quantity known as the "entanglement entropy", capable of capturing changes of quantum behavior of many-body systems. No need to overthink about its definition. Just focus on the fact that the phase diagram displays "domes" (labeled Checkerboard, Star, Rombic, etc ). 

These domes correspond to what we call different "phases", that describe characteristic excitation patterns of the square lattice of qubits whenever parameters of the system lie inside a given dome. These excitation patterns correspond to what is the ground state, the quantum state of lowest energy, for the system. Let's learn how to access these states via a quantum algorithm!

### 3b. Preparing checkerboards with an adiabatic protocol

If we can easily access the ground state in any region of this phase diagram, we can rely on the adiabatic theorem to transform that state into the ground state of any other phase.

> the adiabatic theorem states, in general lines, that under slow enough modifications of the parameters of a quantum system, its ground state can be smoothly transformed while remaining the lowest energy state. Slow enough is defined her as in comparison with the energy gap (divided by $\hbar$, Planck's reduced constant) between the ground state and the lowest excited state

As it turns out, for large negative values for the detuning $\Delta$, the ground state is indeed just a product $| 000.000 \rangle$. A quantum algorithm transforms a given state, typically this product all-0 state, into a target state that answers a computational question, or into a superposition state whose highest weight goes for the state that solves our question. So here is how we pick waveforms that use the adiabatic theorem to prepare a checkerboard state of the Rydberg-atom square lattice:

1. start with large negative $\Delta$ and, as mandated by the quantum hardware, $\Omega = 0$.
2. ramp up $\Omega$ to a given y-cut on the phase diagram that positions you at the "height" of the checkerboard dome. For example, we can pick $\Omega$ so that we are at the dotter horizontal line at $R_B/a\approx 1.2$.
3. ramp $\Delta$ slowly from negative to positive through a trajectory - which can be taken linear for simplicity - that connects us to a target value in the $x$-axis, such as $\frac{\Delta}{\Omega}\approx 2.7$. This will put us exactly at the crossing of the dashed and dotter lines in the phase diagram above!

That is it! Once this process is done, one ramps the Rabi amplitude waveform down, as is mandatory, and measured. At the end of this protocol, if we chose our numbers correctly, we should have prepared a state with high likelihood of corresponding to a checkerboard state.

Let's do that with Bloqade. First, let's pick relevant parameters


In [6]:
# Section imports
from bloqade.atom_arrangement import Square
import numpy as np
from bokeh.io import output_notebook
output_notebook()

# Change the lattice spacing to vary the atom separation a, and thus also Rb/a
delta_end=2*np.pi*6.8 #final detuning
omega_max=2*np.pi*2.5 #max Rabi amplitude
sweep_time = 2.4 #time length of program 
lattice_spacing = 7.0 #size of edges of square lattice
L=3 # number of unit cells in each direction

C6 = 2*np.pi * 862690;
Rb = (C6 / (omega_max) )** (1/6) # R_B during bulk of protocol
print("Rb/a: ",Rb/lattice_spacing)


print("Delta/Omega: ", delta_end/omega_max)

Loading BokehJS ...

Rb/a:  1.1964312624669644
Delta/Omega:  2.72


Now we can do our standard process for creating a program. Notice that, in this case, we do not care about every step of the time-dependent evolution, but only the final state. So the process is much simpler and we can forget batching.

> note a subtlety about Bloqade. While `start` is necessary to begin including atoms to the register via `add_position()`, if we use a formal pre-defined geometry object from within Bloqade, such as `Square()` below, we skip the `start` object!

In [3]:
rabi_amplitude_values = [0.0, omega_max, omega_max, 0.0]
rabi_detuning_values = [-delta_end, -delta_end, delta_end, delta_end]
durations = [0.8, sweep_time, 0.8]

prog = (
    Square(L, lattice_spacing=lattice_spacing)
    .rydberg.rabi.amplitude.uniform.piecewise_linear(durations, rabi_amplitude_values)
    .detuning.uniform.piecewise_linear(durations, rabi_detuning_values)
)


Finally, run and report as usual!

In [4]:
emu_prog=prog.bloqade.python().run(shots=100)
emu_prog.report().show()

Looks like our protocol is pretty good! In 100 shots, we recover the expected checkerboard pattern! 

This success will certainly decrease if we scale the system size (not recommended without using an actual quantum computer!). In that case, gap sizes will reduce and the protocol will have to be adjusted for longer times, or even optimized to make sure that the adiabatic passage is done slowly enough.

# Conclusions
This marks the end of the second chapter. Congratulations as now you are familiarized with the basic phenomenology of neutral-atom quantum computers. 
Revisiting again our learning objectives, now you are able to
- Describe the Rydberg bloqade phenomenon
- Compute the dynamics of multi-qubit Rydberg systems describing the effects of the blockade on Rabi oscillations
- Analyze Rydberg phases on square lattices